In [ ]:
#using Markdown; Markdown.parse_file("README.md")

In [ ]:
MODEL_PATH_KEY = :wNB;
const FLTYPE = Float32;
include("heavyloadings.jl");

In [ ]:
half_b = load(MODEL_FOLDER*"\\bunny\\half_bunny_unmod.ply");
half_b_rot = load(MODEL_FOLDER*"\\bunny\\half_bunny_rot1.ply");

In [ ]:
include("firsticp.jl")

In [ ]:
half_b_sc = scaleMesh(half_b,10);
half_b_rot_sc = scaleMesh(half_b_rot,10);

In [ ]:
vis = Visualizer();

In [ ]:
open(vis)
#IJuliaCell(vis)

In [ ]:
placeMeshInLife(vis,half_b_sc,"hbunny","darkred");

In [ ]:
placeMeshInLife(vis,half_b_rot_sc,"rotbunny","olivedrab2")

## Old workflow with \ and fix Array type

```julia
function firstICP(ref_mesh, read_mesh, iternum; fltype = Float32, sampleN = 50, chopN = 20, v = true)
    #Matrixes
    ref_mat = convert2HomCoordMatrix(vertices(ref_mesh), fltype)
    red_mat = convert2HomCoordMatrix(vertices(read_mesh), fltype)
    #Transposed reference matrix
    ref_mtp = giveTranspose(ref_mat)
    red_mtp = giveTranspose(red_mat)

    #Sampling random indexes for the reference and reading arrays
    #si_ref = randomSampleIndexes(50, ref_mtp, 2)
    #si_red = randomSampleIndexes(50, red_mtp, 2)

    #Building KD tree for the whole ref array (transposed matrix must be used)
    kdtree_ref = KDTree(ref_mtp);
    
    firstDict, ujar = basicIteration(red_mtp, kdtree_ref, ref_mat, iternum, sampleN=sampleN, chopN=chopN, v=v);
    return firstDict, red_mtp, ujar
end

function basicIteration(read_vert, kdtree_r, ref_mhor, itnum; sampleN = 50, chopN = 20, v = true)
    trDict = Dict{Int,Any}()
    for i in 1:itnum
        #selecting random % of the reading
        si_red = randomSampleIndexes(sampleN, read_vert, 2);
        #pairing the points in the two meshes
        indM, traitM = createKnnPairArray(read_vert[:,si_red], kdtree_r);
        #sorting the indexes by the distances in the trait matrix
        s_indM, s_it = sortIndexes(indM, traitM[:,2]);
        #remove the worst %
        ch_ind, ch_it = chopEndOfArray(chopN, s_indM,s_it);
        #make the transformation matrix
        coordTR = transpose(read_vert[:,ch_ind[:,2]])\ref_mhor[ch_ind[:,1],:];
        #transzforálmi az mtp-t kell
        coordTR[:,4] = [0,0,0,1]
        read_vert = transpose(coordTR) * read_vert;
        trDict[i] = coordTR
        if v && i%100 == 0
            print("\r $i'th iteration")
        end
    end
    return trDict, read_vert
end
```

## New workflow
Using arrays (bármilyen lehessen: SArray, CuArray, Array)

Ötlet: amíg lehet csak az az indexeket használni és csak ahol muszáj, ott számolni a konkrét mátrixokkal
### Workflow lépései:
1. [x] reading és ref mesh-t tömbbé alakítani
1. [x] a teljes referencia tömbből kd-tree
### Iteráció
1. [x] reading tömbből sampling
1. [x] samplinged tömbhöz párkeresés a fából
1. ##### Worst `x`% rejection
  * [x] párok sorbarendezése távolság szerint
  * [x] a sorbarendezett párok `x`%-nak kidobása
1. ##### 2.5sigma rejection
  * [ ] szórás számítása
  * [ ] 2.5 sigmánál nagyobbak kidobása
1. [ ] kvaternió magic
1. [ ] transzformáció a megoldással
1. [ ] transzformáció mentése vizualizációhoz
1. [ ] kilépési feltétel check

### Iteráció vége

11. [ ] place in MeshCat
12. [ ] animate in MeshCat


## TODOs:
* újraírni a számolást
* vizualizáció
* animáció

### Newnew workflow with matrix instead of array of array

In [ ]:
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);

#Creating matrixes
ref_mat = makeMatrixFromMesh(vertices(reference_pc));
red_mat = makeMatrixFromMesh(vertices(reading_pc));

#Transposed reference matrix
ref_mtp = giveTranspose(ref_mat)
red_mtp = giveTranspose(red_mat)

#Building KD tree for the whole ref array (transposed matrix must be used)
kdtree_ref = KDTree(ref_mtp);

In [ ]:
si_red = randomSampleIndexes(50, red_mtp, 2);
#pairing the points in the two meshes
indM, traitM = createKnnPairArray(red_mtp[:,si_red], kdtree_ref);
#sorting the indexes by the distances in the trait matrix
dist_arr = @view traitM[:,2]
s_indM, s_it = sortIndexes(indM, dist_arr);
#remove the worst %
ch_ind, ch_it = chopEndOfArray(20, s_indM,s_it);